In [6]:
!pip install --ignore-installed pycaret[full]
!pip install python-dotenv
!pip install s3fs

zsh:1: no matches found: pycaret[full]
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [7]:
# import libraries
import importlib
import boto3, os
from sagemaker import get_execution_role
import pandas as pd
from dotenv import load_dotenv
from load_data import load_data


my_region = boto3.session.Session().region_name

In [50]:
import numpy as np

In [5]:
load_dotenv(".env")
role = get_execution_role()

data_location_s3 = os.getenv("data_location_s3")
target = os.getenv("target")
algorithm_choice = os.getenv("algorithm_choice")
data_location = 's3://{}'.format(data_location_s3)
print(data_location)
print(target)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/kong.nopwattanapong/Library/Application Support/sagemaker/config.yaml


TokenRetrievalError: Error when retrieving token from sso: Token has expired and refresh failed

In [65]:
data = load_data(data_location)
df = data.copy()
df.shape

(7127, 5)

In [66]:
print(df.shape)
# Randomly shuffle the DataFrame
df_shuffled = df.sample(frac=1).reset_index(drop=True)
# Sort by day and then pick the first 80% as your test data. 
train_size = int(0.8 * len(df))
train_data = df_shuffled[:train_size]
test_data = df_shuffled[train_size:]
print(train_data.shape, test_data.shape)

(7127, 5)
(5701, 5) (1426, 5)


In [8]:
from pycaret.datasets import get_data
data = get_data('diabetes')

,Number of times pregnant,Plasma glucose concentration a 2 hours in an oral glucose tolerance test,Diastolic blood pressure (mm Hg),Triceps skin fold thickness (mm),2-Hour serum insulin (mu U/ml),Body mass index (weight in kg/(height in m)^2),Diabetes pedigree function,Age (years),Class variable
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [45]:
algorithm_choice = 'classification'

In [46]:
# Import Pycaret Module
pycaret = importlib.import_module(f"pycaret.{algorithm_choice}")

# # Use the globals() function to update the current namespace with the package's content (But Not Recommended)
# globals().update(vars(pycaret))

In [47]:
s = pycaret.setup(data = data, target = "Class variable", session_id=123)

,Description,Value
0,Session id,123
1,Target,Class variable
2,Target type,Binary
3,Original data shape,"(768, 9)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(537, 9)"
6,Transformed test set shape,"(231, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


In [52]:
pycaret.compare_models()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
dt = pycaret.create_model('lr', n_jobs = 5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8704,0.9008,0.7368,0.8750,0.8000,0.7051,0.7108
1,0.8148,0.7880,0.6316,0.8000,0.7059,0.5735,0.5820
2,0.8704,0.9308,0.6842,0.9286,0.7879,0.6976,0.7145
3,0.6852,0.7744,0.4737,0.5625,0.5143,0.2839,0.2862
4,0.8704,0.9128,0.6842,0.9286,0.7879,0.6976,0.7145
5,0.6481,0.6722,0.3684,0.5000,0.4242,0.1792,0.1835
6,0.7407,0.8030,0.5789,0.6471,0.6111,0.4176,0.4190
7,0.7736,0.8730,0.4444,0.8000,0.5714,0.4342,0.4688
8,0.6604,0.6810,0.4444,0.5000,0.4706,0.2219,0.2227


In [54]:
print(dt)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=5, penalty='l2', random_state=123,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)


In [24]:
print(np.random.randint(1, (len(data.columns)*.85),20))

[6 3 5 3 2 4 3 4 2 2 1 2 2 1 1 2 4 6 5 1]


In [55]:
params = {"max_depth": [2, 2, 2],
          "max_features": [2, 2, 2],
          "min_samples_leaf": [2, 2, 2],
          "bootstrap": [True, True, True]}
tuned_dt = pycaret.tune_model(dt)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8704,0.8992,0.7368,0.8750,0.8000,0.7051,0.7108
1,0.8148,0.7850,0.6316,0.8000,0.7059,0.5735,0.5820
2,0.8704,0.9353,0.6842,0.9286,0.7879,0.6976,0.7145
3,0.7222,0.7744,0.4737,0.6429,0.5455,0.3520,0.3605
4,0.8704,0.9113,0.6842,0.9286,0.7879,0.6976,0.7145
5,0.6481,0.6662,0.3684,0.5000,0.4242,0.1792,0.1835
6,0.7407,0.8030,0.5789,0.6471,0.6111,0.4176,0.4190
7,0.7736,0.8746,0.4444,0.8000,0.5714,0.4342,0.4688
8,0.6604,0.6778,0.4444,0.5000,0.4706,0.2219,0.2227


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [59]:
print(dir(tuned_dt))

['C', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_check_feature_names', '_check_n_features', '_estimator_type', '_get_param_names', '_get_tags', '_more_tags', '_parameter_constraints', '_predict_proba_lr', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_validate_data', '_validate_params', 'class_weight', 'classes_', 'coef_', 'decision_function', 'densify', 'dual', 'feature_names_in_', 'fit', 'fit_intercept', 'get_params', 'intercept_', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_features_in_', 'n_iter_', 'n_jobs', 'penalty', 'predict', 'predict_log_proba', 'predict_proba', 'random_state', 'score', 'set_params',

In [61]:
tuned_dt.get_params()

{'C': 7.863,
 'class_weight': {},
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 1000,
 'multi_class': 'auto',
 'n_jobs': 5,
 'penalty': 'l2',
 'random_state': 123,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [57]:
pycaret.evaluate_model(tuned_dt)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [57]:
pycaret.predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.7322,1.6287,1.2762,0.9706,0.0757,0.0499


,Bakerloo,dayOfWeek,hour,minute,Bakerloo10,prediction_label
2676,28,1,17,25,25.0,27.382988
724,2,2,23,49,5.0,5.354857
4595,28,1,17,26,28.0,27.378793
1112,22,6,16,12,21.0,22.482936
4064,19,5,13,30,19.0,19.347143
...,...,...,...,...,...,...
3854,24,2,10,12,23.0,23.665934
4481,22,4,10,50,21.0,21.878490
467,27,0,17,44,25.0,25.506182
3551,18,0,22,48,19.0,18.555363


In [58]:
final_best_model = pycaret.finalize_model(best)
print(final_best_model)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Bakerloo', 'dayOfWeek', 'hour',
                                             'minute'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=[],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('actual_estimator',
                 <catboost.core.CatBoostRegressor object at 0x2b94e70a0>)])


In [59]:
final_prediction = pycaret.predict_model(final_best_model, data=test_data)
final_prediction.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,CatBoost Regressor,0.6974,1.1071,1.0522,0.9808,0.0656,0.0493


,Bakerloo,dayOfWeek,hour,minute,Bakerloo10,prediction_label
5701,23,1,20,3,24.0,23.012273
5702,23,0,10,22,23.0,22.525582
5703,1,1,3,34,1.0,0.940797
5704,25,0,8,53,27.0,26.304840
5705,23,1,18,59,23.0,24.204963


In [60]:
from pycaret.utils.generic import check_metric
check_metric(final_prediction.Bakerloo10, final_prediction.prediction_label, 'RMSE')

1.0522

In [61]:
# trying to save the model
from pycaret.regression import save_model

save_model(final_best_model, 'final_best_model')


Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['Bakerloo', 'dayOfWeek', 'hour',
                                              'minute'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('actual_estimator',
                  <catboost.core.CatBoostRegressor object at 0x2b94e70a0>)]),
 'final_best_model.pkl')

In [62]:
import tarfile

with tarfile.open('final_best_model.tar.gz', 'w:gz') as tar:
    tar.add('final_best_model.pkl')

In [21]:
import boto3

s3 = boto3.client('s3')
s3.upload_file('final_best_model.tar.gz', 'mlops-feature-engineering', 'final_best_model.tar.gz')


S3UploadFailedError: Failed to upload final_best_model.tar.gz to mlops-feature-engineering/final_best_model.tar.gz: An error occurred (AccessDenied) when calling the PutObject operation: User: arn:aws:sts::135544376709:assumed-role/test-model-29-09-23-sagemaker-role/SageMaker is not authorized to perform: kms:GenerateDataKey on resource: arn:aws:kms:eu-west-1:135544376709:key/1c31519d-0b3d-4b45-a414-ed01283350e0 because no identity-based policy allows the kms:GenerateDataKey action

S3UploadFailedError: Failed to upload final_best_model.tar.gz to mlops-feature-engineering/final_best_model.tar.gz: An error occurred (AccessDenied) when calling the PutObject operation: User: arn:aws:sts::135544376709:assumed-role/test-model-29-09-23-sagemaker-role/SageMaker is not authorized to perform: kms:GenerateDataKey on resource: arn:aws:kms:eu-west-1:135544376709:key/1c31519d-0b3d-4b45-a414-ed01283350e0 because no identity-based policy allows the kms:GenerateDataKey action

In [19]:
from sagemaker.model import Model
model_data = 'file:///home/ec2-user/SageMaker/final_best_model.tar.gz'
# Move model to s3. Member must satisfy regular expression pattern: ^(https|s3)://([^/]+)/?(.*)$
model = Model(
    image_uri='135544376709.dkr.ecr.eu-west-1.amazonaws.com/mlops-pycaret-repo:latest',  # The ECR image you pushed
    model_data=model_data,  # Location of your serialized model
    role=role
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge'
)

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: 1 validation error detected: Value 'file:///home/ec2-user/SageMaker/final_best_model.tar.gz' at 'primaryContainer.modelDataUrl' failed to satisfy constraint: Member must satisfy regular expression pattern: ^(https|s3)://([^/]+)/?(.*)$

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: 1 validation error detected: Value 'file:///home/ec2-user/SageMaker/final_best_model.tar.gz' at 'primaryContainer.modelDataUrl' failed to satisfy constraint: Member must satisfy regular expression pattern: ^(https|s3)://([^/]+)/?(.*)$

<li>Feature importance and hyperparameter tuning with pycaret. How can pycaret handle this and do we need to have our step for this separately?</li>
<li>Timebox the custom container until Friday.</li> 
<li>MVP can be done with sagemaker for now. </li> 